# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

## Solution

In [1]:
# Dictionary to store the 2-grams and their frequencies
two_grams = {}

with open('w2_.txt', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        # Split the line by tab
        parts = line.split('\t')
        # The frequency is the first part of the line
        frequency = int(parts[0])
        # The 2-gram is the second part of the line
        two_gram = tuple(parts[1:])
        # Store the 2-gram and its frequency in the dictionary
        two_grams[two_gram] = frequency

# Sort the 2-grams by frequency
two_grams = dict(sorted(two_grams.items(), key=lambda item: item[1], reverse=True))

# Print a sample of the 2-grams
for two_gram, frequency in list(two_grams.items())[:10]:
    print(f'{two_gram}: {frequency}')

('of', 'the\n'): 2586813
('in', 'the\n'): 2043262
('to', 'the\n'): 1055301
('on', 'the\n'): 920079
('and', 'the\n'): 737714
('to', 'be\n'): 657504
('at', 'the\n'): 617976
('for', 'the\n'): 616400
('in', 'a\n'): 544137
('do', "n't\n"): 537718


In [2]:
# Dictionary to store the 5-grams and their frequencies
five_grams = {}

with open('w5_.txt', 'r', encoding='utf-8') as file:
    for line in file.readlines():
        # Split the line by tab
        parts = line.split('\t')
        # The frequency is the first part of the line
        frequency = int(parts[0])
        # The 5-gram is the second part of the line
        five_gram = tuple(parts[1:])
        # Store the 5-gram and its frequency in the dictionary
        five_grams[five_gram] = frequency

# Sort the 5-grams by frequency
five_grams = dict(sorted(five_grams.items(), key=lambda item: item[1], reverse=True))

# Print a sample of the 5-grams
for five_gram, frequency in list(five_grams.items())[:10]:
    print(f'{five_gram}: {frequency}')

('at', 'the', 'end', 'of', 'the\n'): 13588
('i', 'do', "n't", 'want', 'to\n'): 12744
('in', 'the', 'middle', 'of', 'the\n'): 9124
('i', 'do', "n't", 'know', 'what\n'): 8076
('you', 'do', "n't", 'have', 'to\n'): 7186
('i', 'do', "n't", 'know', 'if\n'): 6455
('for', 'the', 'first', 'time', 'in\n'): 6006
('i', 'do', "n't", 'think', 'it\n'): 5559
('there', 'are', 'a', 'lot', 'of\n'): 5523
('i', 'do', "n't", 'think', 'that\n'): 5466


In [3]:
def calculate_conditional_probabilities(ngrams):
    """
    Calculate conditional probabilities for n-grams from their frequencies.
    """
    # For storing the total frequency of each prefix (n-1 gram)
    prefix_counts = {}
    
    # For storing the conditional probabilities of each n-gram
    conditional_probabilities = {}
    
    for ngram, frequency in ngrams.items():
        # For 2-grams, the prefix is just the first word
        # For 5-grams, it's the first 4 words
        prefix = ngram[:-1]
        # Update the prefix frequency
        if prefix in prefix_counts:
            prefix_counts[prefix] += frequency
        else:
            prefix_counts[prefix] = frequency
    
    for ngram, frequency in ngrams.items():
        prefix = ngram[:-1]
        # Calculate the conditional probability
        conditional_probability = frequency / prefix_counts[prefix]
        conditional_probabilities[ngram] = conditional_probability
    
    return conditional_probabilities

def train_ngram_model(two_grams, five_grams):
    """
    Train an N-gram model.
    """
    # Calculate conditional probabilities for 2-grams and 5-grams
    two_gram_probabilities = calculate_conditional_probabilities(two_grams)
    five_gram_probabilities = calculate_conditional_probabilities(five_grams)
    
    # Combine the probabilities into a single model
    ngram_model = {
        '2-grams': two_gram_probabilities,
        '5-grams': five_gram_probabilities,
    }
    
    return ngram_model

# Train the N-gram model using the previously loaded data
ngram_model = train_ngram_model(two_grams, five_grams)

# Print some of the probabilities to verify
print("Sample 2-gram probabilities:")
for ngram, prob in list(ngram_model['2-grams'].items())[:10]:
    print(f"{ngram}: {prob}")

print("\nSample 5-gram probabilities:")
for ngram, prob in list(ngram_model['5-grams'].items())[:10]:
    print(f"{ngram}: {prob}")

Sample 2-gram probabilities:
('of', 'the\n'): 0.24906581532057098
('in', 'the\n'): 0.28760887056135515
('to', 'the\n'): 0.10137702785084556
('on', 'the\n'): 0.3516112155875741
('and', 'the\n'): 0.06932882289467387
('to', 'be\n'): 0.06316283346651083
('at', 'the\n'): 0.33653104701161896
('for', 'the\n'): 0.18615500817070693
('in', 'a\n'): 0.07659254075133004
('do', "n't\n"): 0.4186035287010569

Sample 5-gram probabilities:
('at', 'the', 'end', 'of', 'the\n'): 0.5994617726209909
('i', 'do', "n't", 'want', 'to\n'): 0.707724773699117
('in', 'the', 'middle', 'of', 'the\n'): 0.5612351602386664
('i', 'do', "n't", 'know', 'what\n'): 0.23234270260939613
('you', 'do', "n't", 'have', 'to\n'): 0.5879080422154954
('i', 'do', "n't", 'know', 'if\n'): 0.18570729882908024
('for', 'the', 'first', 'time', 'in\n'): 0.4382661996497373
('i', 'do', "n't", 'think', 'it\n'): 0.1372762069391283
('there', 'are', 'a', 'lot', 'of\n'): 0.9684376643871646
('i', 'do', "n't", 'think', 'that\n'): 0.13497962711445857


In [4]:
def tokenize(text):
    return text.split()

def is_misspelled(word, ngram_model):
    return not any(word in model for model in ngram_model.values())

def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts    = [L + c + R     for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def correct_word_contextual(word, left_context, right_context, ngram_model):
    # Generate possible corrections for the misspelled word
    candidates = set(edits1(word)) | set(edits2(word))

    # Filter out non-existing words
    valid_candidates = [w for w in candidates if not is_misspelled(w, ngram_model)]
    if not valid_candidates:
        valid_candidates = [word]  # fallback to the original word if no candidates found

    # Prepare contexts for comparison
    contexts = []
    if left_context:
        contexts += [(left_context[-1],)]  # 2-gram context
    if len(left_context) >= 4:
        contexts += [(tuple(left_context[-4:]),)]  # 5-gram context

    # Choose the best candidate based on the context
    best_score = 0
    best_candidate = word  # Default to original word
    for candidate in valid_candidates:
        candidate_score = 0
        for context in contexts:
            if context + (candidate,) in ngram_model['2-grams']:
                candidate_score += ngram_model['2-grams'][context + (candidate,)]
            if context + (candidate,) in ngram_model['5-grams']:
                candidate_score += ngram_model['5-grams'][context + (candidate,)]
        if candidate_score > best_score:
            best_score = candidate_score
            best_candidate = candidate

    return best_candidate



In [5]:
def correct_text_contextual(text_line, ngram_model):
    words = tokenize(text_line)
    corrected_words = []

    for i, word in enumerate(words):
        if is_misspelled(word, ngram_model):
            left_context = words[max(0, i - 4):i]
            right_context = words[i + 1:i + 5]
            corrected_word = correct_word_contextual(word, left_context, right_context, ngram_model)
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    return ' '.join(corrected_words)

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## Justifications for Implementation Choices

### Choice of N-gram Dataset

- **2-grams and 5-grams**: The use of both 2-grams and 5-grams strikes an optimal balance between leveraging immediate context and understanding broader textual structures. 2-grams facilitate quick, context-aware corrections for common mistakes by focusing on the immediate predecessor of a word. Conversely, 5-grams provide deeper insights into the text structure, essential for resolving more complex errors requiring an understanding of extended textual contexts.

### Weights for Edit Distances

- **Edit1 Prioritization**: Favoring corrections that are one edit distance away before considering two-edit distance corrections reflects a balance between correction accuracy and computational performance. This approach is rooted in the observation that most typing mistakes result in words that are a single edit distance away from their correct form, thus prioritizing these corrections reduces the computational load by avoiding the processing of a broader set of less likely candidates from `edits2`.

### Efficiency and Scalability

- **Tokenization and Contextual Evaluation**: Adopting a word-by-word processing strategy not only mirrors principles found in natural language processing (NLP) but also ensures scalability. This method allows the system to handle texts of varying lengths efficiently, from short messages to comprehensive documents.


## Evaluate on a test set

First, let's add the implementation of norvig's corrector so that we can compare the results.

In [6]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def norvig_correct_text(sentence):

    # Tokenize the sentence into words
    words_in_sentence = words(sentence)  # Using the 'words' function from Norvig's implementation
    
    # Apply the 'correction' function to each word
    corrected_words = [correction(word) for word in words_in_sentence]
    
    # Reassemble the sentence from the corrected words
    corrected_sentence = ' '.join(corrected_words)
    
    return corrected_sentence

In [7]:
import random

# Generating a sample testset
test_sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "An apple a day keeps the doctor away.",
    "Better late than never, but never late is better.",
    "Actions speak louder than words.",
    "Beauty is in the eye of the beholder."
]

# Function to introduce errors in the sentences
def introduce_errors(sentence, noise_probability=0.1):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    words = sentence.split()
    error_words = []

    for word in words:
        if random.random() < noise_probability:
            error_type = random.choice(['replace', 'delete', 'insert', 'transpose'])
            if error_type == 'replace' and len(word) > 1:
                idx = random.randint(0, len(word) - 1)
                word = word[:idx] + random.choice(letters) + word[idx + 1:]
            elif error_type == 'delete' and len(word) > 1:
                idx = random.randint(0, len(word) - 1)
                word = word[:idx] + word[idx + 1:]
            elif error_type == 'insert':
                idx = random.randint(0, len(word) - 1)
                word = word[:idx] + random.choice(letters) + word[idx:]
            elif error_type == 'transpose' and len(word) > 1:
                idx = random.randint(0, len(word) - 2)
                word = word[:idx] + word[idx+1] + word[idx] + word[idx+2:]
        error_words.append(word)

    return ' '.join(error_words)

In [8]:
def evaluate_accuracy(original_sentences, corrected_sentences):
    # A simple function to calculate the accuracy of corrected sentences against the originals
    correct = 0
    total = sum(len(sentence.split()) for sentence in original_sentences)
    
    for original, corrected in zip(original_sentences, corrected_sentences):
        original_words = original.split()
        corrected_words = corrected.split()
        
        correct += sum(o == c for o, c in zip(original_words, corrected_words))
    
    return correct / total

# Introduce errors into the test set
noisy_sentences = [introduce_errors(sentence, noise_probability=0.2) for sentence in test_sentences]

# Correct the errors using both correctors
corrected_sentences_context = [correct_text_contextual(sentence, ngram_model) for sentence in noisy_sentences]
corrected_sentences_norvig = [norvig_correct_text(sentence) for sentence in noisy_sentences] # Assume a function exists

# Evaluate accuracies
accuracy_context = evaluate_accuracy(test_sentences, corrected_sentences_context)
accuracy_norvig = evaluate_accuracy(test_sentences, corrected_sentences_norvig)

print(f"Context-sensitive Corrector Accuracy: {accuracy_context}")
print(f"Norvig's Corrector Accuracy: {accuracy_norvig}")

Context-sensitive Corrector Accuracy: 0.8205128205128205
Norvig's Corrector Accuracy: 0.6666666666666666
